In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
ws.write_config(path='.azureml')
exp = Experiment(workspace=ws, name="quick-starts-ws-165838")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-167504
Azure region: southcentralus
Subscription id: 9a7511b8-150f-4a58-8528-3e7d50216c31
Resource group: aml-quickstarts-167504


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
amlcompute_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    aml_compute = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    aml_compute = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

aml_compute.wait_for_completion(show_output=True)

InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, quniform
from azureml.core import ScriptRunConfig
from azureml.core import Environment
import os

# Specify parameter sampler
ps = RandomParameterSampling({'--C':uniform(0.1,1),
'--max_iter':quniform(100,1500,100)}

)

# Specify a Policy
policy = early_termination_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)


if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py

script_folder='./training'
os.makedirs(script_folder, exist_ok=True)
import shutil
shutil.copy('./train.py', script_folder)

env=Environment.get(workspace=ws, name="AzureML-Tutorial")

src= ScriptRunConfig(source_directory= script_folder,
                    script='train.py',
                    compute_target= aml_compute,
                     environment=env
                    )

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config =HyperDriveConfig(run_config=src, 
                             hyperparameter_sampling=ps,
                             policy=policy,
                             primary_metric_name="Accuracy", 
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                             max_total_runs=15,
                             max_concurrent_runs=3)

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hdr=exp.submit(config=hyperdrive_config)

In [5]:
RunDetails(hdr).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [7]:
import joblib
# Get your best run and save the model from that run.
best_run=hdr.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguments']) 
print(best_run.get_file_names())

model=best_run.register_model(model_name="Best", model_path=".")

best_run_metrics= best_run.get_metrics()
print('\n Accuracy: ', best_run_metrics['Accuracy'])

### YOUR CODE HERE ###

['--C', '0.8585154790911844', '--max_iter', '900']
['logs/azureml/19_azureml.log', 'logs/azureml/dataprep/backgroundProcess.log', 'logs/azureml/dataprep/backgroundProcess_Telemetry.log', 'logs/azureml/dataprep/rslex.log', 'outputs/model.joblib', 'system_logs/cs_capability/cs-capability.log', 'system_logs/hosttools_capability/hosttools-capability.log', 'system_logs/lifecycler/execution-wrapper.log', 'system_logs/lifecycler/lifecycler.log', 'system_logs/lifecycler/vm-bootstrapper.log', 'user_logs/std_log.txt']

 Accuracy:  0.9101669195751139


In [8]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
ds=TabularDatasetFactory.from_delimited_files(["https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"])

In [9]:
from train import clean_data
from sklearn.model_selection import train_test_split
import pandas as pd

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

#Split into train and test datasets
x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=0.20)

train_df=pd.concat([x_train, y_train], axis=1)
test_df=pd.concat([x_test, y_test], axis=1)

train_df.head()




,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,y
11613,46,1,0,1,0,11,1,622,1,999,...,0,0,0,0,1,0,0,0,0,0
5175,39,1,0,1,0,7,1,173,3,999,...,1,0,0,0,1,0,0,0,0,0
4186,29,0,0,1,0,8,3,875,2,999,...,0,0,0,0,0,0,0,1,0,1
16603,65,1,0,0,0,5,1,1076,3,6,...,1,0,0,0,0,0,0,1,0,1
21991,34,0,0,0,0,11,2,386,1,999,...,0,0,0,1,0,0,0,0,0,0


In [17]:
#save the dataframe as csv and upload to datastore
if not os.path.isdir('training'):
    os.mkdir('training')
pd.DataFrame(train_df).to_csv("training/train_df.csv", index=False)
pd.DataFrame(test_df).to_csv("training/test_df.csv", index=False)

data_store=ws.get_default_datastore()
data_store.upload(src_dir='./training', target_path='bankmarketing', overwrite=True, show_progress=True)

Uploading an estimated of 6 files
Uploading ./training/.amlignore
Uploaded ./training/.amlignore, 1 files out of an estimated total of 6
Uploading ./training/.amlignore.amltmp
Uploaded ./training/.amlignore.amltmp, 2 files out of an estimated total of 6
Uploading ./training/test_df
Uploaded ./training/test_df, 3 files out of an estimated total of 6
Uploading ./training/test_df.csv
Uploaded ./training/test_df.csv, 4 files out of an estimated total of 6
Uploading ./training/train.py
Uploaded ./training/train.py, 5 files out of an estimated total of 6
Uploading ./training/train_df.csv
Uploaded ./training/train_df.csv, 6 files out of an estimated total of 6
Uploaded 6 files


$AZUREML_DATAREFERENCE_e8d6aaa35b2447efb9e1f1c6adc1b915

In [19]:
from azureml.core.dataset import Dataset

train_df=Dataset.Tabular.from_delimited_files(path=data_store.path('bankmarketing/train_df.csv'))

In [20]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=train_df,
    label_column_name='y',
    n_cross_validations=5,
    compute_target=aml_compute 
    )

In [21]:
# Submit your automl run

### YOUR CODE HERE ###
auto_exp=Experiment(workspace=ws, name="auto_exp")
automl_run=exp.submit(automl_config, show_output=False)
RunDetails(automl_run).show()
automl_run.wait_for_completion()

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
quick-starts-ws-165838,AutoML_18a9277d-615e-4e5e-a91f-389ba88aa034,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###